<a href="https://colab.research.google.com/github/jachiaram/GuitarML/blob/main/GuitarML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Guitar Machine Learning Note Classifier Model V1**

## **Install Dependencies:**
In this section tensorflow is install if it isn't already present and other dependencies used in the notebook are imported here.

In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tensorflow-io matplotlib

In [1]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

## **Setting up Training and Testing Data for Model:**
In this section the csv file is read in and converted to a dataframe. Then that is split into X and y data, which is then split into training and testing data.

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/jachiaram/GuitarML/main/Guitar_features.csv')

In [14]:
y_column = 'note_names'
x_columns = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'zero_crossing_rate', 'tonnetz']

X = df[x_columns]
y = df[y_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


label_encoder = LabelEncoder()
y_train_numeric = label_encoder.fit_transform(y_train)
y_test_numeric = label_encoder.fit_transform(y_test)

y_train_one_hot = to_categorical(y_train_numeric, num_classes=37)

In [15]:
X_train.head()

,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,zero_crossing_rate,tonnetz
1363,0.278707,700.276462,1665.780273,987.718665,0.009787,0.044815
782,0.320638,1022.973373,1922.030725,1700.186821,0.014733,-0.019958
1982,0.217635,628.676797,1801.778931,664.720618,0.012600,0.077506
2086,0.184333,2395.893209,2143.029510,3520.678711,0.088899,0.030428
1353,0.298975,1399.884443,2049.209154,1586.903448,0.058488,0.032899


In [16]:
y_train.head()

1363    18. A1
782     20. B1
1982    13. E1
2086    30. A2
1353    18. A1
Name: note_names, dtype: object

## **Sequential Model Testing:**
In this section the model will be created, for the time being a sequential model is being used. The layers will need to be manipulated until a higher accuracy score is achieved (the goal is above 75%).


In [17]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [18]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=37, activation='softmax'))

In [19]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train_one_hot, epochs=50, batch_size=10)

Epoch 1/50
182/182 [==============================] - 1s 6ms/step - loss: 3.6345 - accuracy: 0.0281
Epoch 2/50
182/182 [==============================] - 1s 6ms/step - loss: 3.6826 - accuracy: 0.0264
Epoch 3/50
182/182 [==============================] - 1s 5ms/step - loss: 3.6186 - accuracy: 0.0270
Epoch 4/50
182/182 [==============================] - 1s 5ms/step - loss: 3.6109 - accuracy: 0.0264
Epoch 5/50
182/182 [==============================] - 1s 4ms/step - loss: 3.6107 - accuracy: 0.0275
Epoch 6/50
182/182 [==============================] - 1s 3ms/step - loss: 3.6106 - accuracy: 0.0303
Epoch 7/50
182/182 [==============================] - 1s 4ms/step - loss: 3.6104 - accuracy: 0.0303
Epoch 8/50
182/182 [==============================] - 1s 4ms/step - loss: 3.6103 - accuracy: 0.0303
Epoch 9/50
182/182 [==============================] - 1s 3ms/step - loss: 3.6102 - accuracy: 0.0303
Epoch 10/50
182/182 [==============================] - 1s 3ms/step - loss: 3.6102 - accuracy: 0.0303

In [22]:
from tensorflow.keras.utils import to_categorical
# Convert categorical test labels to one-hot encoding
y_test_one_hot = to_categorical(label_encoder.transform(y_test), num_classes=37)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

15/15 [==============================] - 0s 2ms/step - loss: 3.6283 - accuracy: 0.0132
Test Loss: 3.6283
Test Accuracy: 0.0132


## **Testing a Different Model Type:**
In this section I was experimenting with a Random Forest Classifier to see if I could get any better results with this model versus the other one.

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your training data and labels
# Assuming X_test and y_test are your test data and labels

In [26]:
# Convert string labels to numeric
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_train_encoded

array([ 9, 12,  4, ..., 33, 33, 14])

In [27]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(max_depth=20, min_samples_split=5, min_samples_leaf=16, max_features='log2', n_estimators=300, random_state=42)

In [28]:
# Train the Random Forest classifier
rf_classifier.fit(X_train, y_train_encoded)

RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=16,
                       min_samples_split=5, n_estimators=300, random_state=42)

In [29]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [30]:
# Convert predictions back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6255506607929515
